In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import logging
from gsheet import *
from mailer import *
from qr_code import *
from payment_data import *


"""
If logging in results in an error, delete the 'token.json' to re-authenticate.
""";

Succesfully read config from: ['./config_prispevky.cfg']


In [4]:
logging.basicConfig(
    format="[%(levelname)s] %(message)s",
    level=logging.INFO
)

In [5]:
sheet = Sheets(SPREADSHEET_ID)

[INFO] file_cache is only supported with oauth2client<4.0.0


In [6]:
# sheet_names = sheet.get_sheet_names()
# sheet_name = '4pvs_all'
# sheet_name = 'PLATBY_PODZIM_2022_filtered'
# sheet_name="ExportNoFormulas"
# sheet_name="SkautisExport20231107"
sheet_name="PlatbyPodzim2024"
# sheet_name = 'TESTv2'
if 'TEST' in sheet_name:
    testmode=True
else:
    testmode=False
df = sheet.get_dataframe(sheet_name)

In [7]:
df

,Jméno,Příjmení,Příjmení jméno,Osoba,Poplatek,Zaplaceno,Registrační číslo,Jednotka,Přezdívka,,...,Otec: mail,Matka: mail,Ostatní: mail,Mobil (další),Telefon (další),Telefon / mobil (hlavní),Telefon do práce,Otec: telefon,Matka: telefon,Ostatní: telefon
0,Karel,Dvořák,Dvořák Karel,Dvořák Karel (Šárlí),1000,0,10001,Albatrosové,Šárlí,(Šárlí),...,,,,,,721978545,None,None,None,None
1,David,Svoboda,Svoboda David,Svoboda David (Cedník),1000,0,10010,Albatrosové,Cedník,(Cedník),...,,,,,,732131483,None,None,None,None
2,Jan,Mrkos,Mrkos Jan,Mrkos Jan (Hondzik),1000,0,10015,Albatrosové,Hondzik,(Hondzik),...,,,,,,720411622,None,None,None,None
3,Oldřich,Milec,Milec Oldřich,Milec Oldřich (Olda),1000,0,10017,Albatrosové,Olda,(Olda),...,,,,,,732598732,None,None,None,None
4,Tomáš,Kopal,Kopal Tomáš,Kopal Tomáš (Kohn),1000,0,10020,Albatrosové,Kohn,(Kohn),...,,,,,,731229603,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,Lada,Zámečníková,Zámečníková Lada,Zámečníková Lada,1000,0,60006,Želvy,,,...,martinz6492@seznam.cz,veronika.zamecnikova@omska.cz,,,,,,737859247,724028382,None
225,Luisa,Šilingerová,Šilingerová Luisa,Šilingerová Luisa,1000,0,60009,Želvy,,,...,frantisek.pekar@email.cz,linda.silingerova@jsns.cz,,,,,,603157305,603290152,None
226,Ema,Brücklerová,Brücklerová Ema,Brücklerová Ema,1000,0,40429,Želvy,,,...,bruckler-jiri@volny.cz,barborabr@centrum.cz,,,,776048148,,608979018,776048148,None
227,Marianna,Fenclová,Fenclová Marianna,Fenclová Marianna,1000,0,40430,Želvy,,,...,dalifen@gmail.com,annamariefenclova@gmail.com,,,,604970901,,603202327,736751580,None


In [8]:
NEEDED_COLS = NeededColumns()
pdf=PaymentsDataFrame(df, NEEDED_COLS)

/mnt/c/Users/mrkos/scth/projects/4pvs_prispevky_qr_kod/payment_data.py:114: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_fixed.loc[:, col] = pd.to_numeric(payments.loc[:, col])
[INFO] Of 229 people, missing email: 2,
	emailable paid: 0
	emailable unpaid: 227.
 Also there were 0 invalid emails.


In [9]:
pdf.df_missing_email

,Osoba,Jednotka,Registrační číslo,E-mail (hlavní),Otec: mail,Matka: mail,E-mail (další),Ostatní: mail,Poplatek,Zaplaceno,valid_addresses,invalid_addresses
22,Makhov Iurii (Jura),Albatrosové,10063,None,None,None,None,None,1000,0,(),()
23,Makhov Roman,Albatrosové,10064,None,None,None,None,None,1000,0,(),()


In [10]:
emails = []

"""
!!!!!WARNING! CURRENTLY SET TO EMAIL ALL AVAILABLE EMAIL ADDRESSES!!!!!

Modify this setting on second line:
    recepient_addresses = row.valid_addresses
    
Modify order in in class NeededColumns
"""

for i, row in pdf.df_emailable_unpaid.iterrows():
    recepient_addresses = row.valid_addresses[:4]
    pi = PaymentInfo.from_df_row(row, NEEDED_COLS)
    email = PaymentEmail(pi, recepient_addresses, testmode=testmode)
    emails.append(email.msg)

In [11]:
pdf.df_emailable_unpaid

,Osoba,Jednotka,Registrační číslo,E-mail (hlavní),Otec: mail,Matka: mail,E-mail (další),Ostatní: mail,Poplatek,Zaplaceno,valid_addresses,invalid_addresses
0,Dvořák Karel (Šárlí),Albatrosové,10001,kareldvorak@centrum.cz,None,None,None,None,1000,0,"(""E-mail (hlavní)"" <kareldvorak@centrum.cz>,)",()
1,Svoboda David (Cedník),Albatrosové,10010,cednik@email.cz,None,None,None,None,1000,0,"(""E-mail (hlavní)"" <cednik@email.cz>,)",()
2,Mrkos Jan (Hondzik),Albatrosové,10015,hondzik@skaut.cz,None,None,"hondzik@skautka.cz, jan.mrkos@skaut.cz, mrkos....",None,1000,0,"(""E-mail (hlavní)"" <hondzik@skaut.cz>, ""E-mail...",()
3,Milec Oldřich (Olda),Albatrosové,10017,oldamilec@seznam.cz,None,None,trallda@skaut.cz,None,1000,0,"(""E-mail (hlavní)"" <oldamilec@seznam.cz>, ""E-m...",()
4,Kopal Tomáš (Kohn),Albatrosové,10020,kopalektomas@seznam.cz,None,None,kohn@skaut.cz,None,1000,0,"(""E-mail (hlavní)"" <kopalektomas@seznam.cz>, ""...",()
...,...,...,...,...,...,...,...,...,...,...,...,...
224,Zámečníková Lada,Želvy,60006,None,martinz6492@seznam.cz,veronika.zamecnikova@omska.cz,None,None,1000,0,"(""Matka: mail"" <veronika.zamecnikova@omska.cz>...",()
225,Šilingerová Luisa,Želvy,60009,None,frantisek.pekar@email.cz,linda.silingerova@jsns.cz,None,None,1000,0,"(""Matka: mail"" <linda.silingerova@jsns.cz>, ""O...",()
226,Brücklerová Ema,Želvy,40429,barborabr@centrum.cz,bruckler-jiri@volny.cz,barborabr@centrum.cz,None,None,1000,0,"(""E-mail (hlavní)"" <barborabr@centrum.cz>, ""Ot...",()
227,Fenclová Marianna,Želvy,40430,None,dalifen@gmail.com,annamariefenclova@gmail.com,None,None,1000,0,"(""Matka: mail"" <annamariefenclova@gmail.com>, ...",()


In [12]:
with open(f'./test.eml', 'wb') as f:
    f.write(bytes(emails[0]))

## ⚡ Danger Zone ⚡

In [13]:
mailer = Mailer()
sent = []

[INFO] file_cache is only supported with oauth2client<4.0.0


In [14]:
for i, email in enumerate(emails[:]):
    print(f'Sending email {i+1}/{len(emails)}:')
    sent_msg = mailer.send_message(email)
    sent.append((email, sent_msg))

[INFO] Message Id: 1924f30615c9bc8e sent.
[INFO] Message Id: 1924f306255945d5 sent.
[INFO] Message Id: 1924f3065b9e768a sent.
[INFO] Message Id: 1924f3067e5513d1 sent.
[INFO] Message Id: 1924f3068bd5e050 sent.
[INFO] Message Id: 1924f306c82139b0 sent.
[INFO] Message Id: 1924f306fa0be0bf sent.
[INFO] Message Id: 1924f30729b8c7a5 sent.
[INFO] Message Id: 1924f30742ad3d3a sent.
[INFO] Message Id: 1924f3077c6c9ff1 sent.
[INFO] Message Id: 1924f3079ed60c48 sent.
[INFO] Message Id: 1924f307dee8e95e sent.
[INFO] Message Id: 1924f307e158d0ed sent.
[INFO] Message Id: 1924f3083bfc7287 sent.
[INFO] Message Id: 1924f30855a309a7 sent.
[INFO] Message Id: 1924f30867d976ab sent.
[INFO] Message Id: 1924f308abf52f5a sent.
[INFO] Message Id: 1924f308ceb0b561 sent.
[INFO] Message Id: 1924f308ea6c4db7 sent.
[INFO] Message Id: 1924f3090244d3f0 sent.
[INFO] Message Id: 1924f309429b9607 sent.
[INFO] Message Id: 1924f3097b66935a sent.
[INFO] Message Id: 1924f30983a36bef sent.
[INFO] Message Id: 1924f309bd7f61e